# Building an ESG index

## Understanding ESG

### What is an ESG index ?

The ESG, or Environmental, Social and Governance, index aims to assess a company, an organization or a country's practices. As early as the 1960s, investors began to exclude from their portfolio certain company that didn't follow evironement or ethical guidelines. Today, with climate deterioration, more and more investors embrace the use of a ESG index to guide their investments.

For example, when ESG indexes were first mention at the UN Principles for Responsible Investment (PRI) in 2006 only 63 investements company (representing \\$6.5 trillion in asset under management) had included an ESG in their financial report. In 2020, 2450 company (\\$80 trillion in AUM) have adopted this practice. 

Futhermore, some reasearch have shown that ESG index could be link with investement return, firm value and profitability. Those results indicate how robust ESG index is crucial in understanding a company's well-being. 

### How to build an ESG index

As the importance of ESG indexing became clear, many companies started to developpe their own ranking to be used by investors and asset managers. Of the many ESG available, MSCI ESG Indices, S&P 500 ESG Index, FTSE4Good Index Series are among the most renounded. 

Each ESG Index must be developped by keeping in mind the three pillars of ESG : 

* Environmental
    * Energy use
    * Waste production
    * Greenhouse gas emissions
    * Carbon footprint
* Social
    * Workplace health and safety
    * Equal employemnent opportunity
    * Fair pay
    * Community engagement
* Governance
    * Ethical business practices
    * Accounting integrity and transparency
    * Corporate governance
    * Avoiding conflicts of interest
    
Most of the popular ESG indices are based upon those principles, the differences in them is the variables and the weight attributed to them. They can differ by the data used wich are often public and/or private corporate, sector or country data.

### Different scale of ESG

Even if their bare the same name and have the same scope, evalutating environmental, social and governance practices and performance, company and country ESG differs by the means and the indicators used to developpe the score.

At a company level, the environnemental score will be assess by analysing carbon emissions, waste management wherease at a country level this will not be possible so it'll be assess using funds allocated to environemental policies, climate change mitigation efforts for example. For the social aspect of the index, human rights records, educational funds will be analyse for a country as oposed to employee diversity, workplace health for a company. As for governance, ethics and transparency will be key for a company while political stability and corruption levels will be used for a country

In the end, the aim is to provide an index useful for investors but at a different scale.

## Computing the Environmental Score